In [2]:
import sys, os


project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [3]:
import pandas as pd
import os
from src.data_loader import load_data, inspect_data , explain, describe_data , save_processed_dataset
from scipy.stats import zscore

In [4]:
df = load_data()

c:\Users\DELL\Desktop\household-power-forecasting\src\data_loader.py:23: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
c:\Users\DELL\Desktop\household-power-forecasting\src\data_loader.py:23: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(
c:\Users\DELL\Desktop\household-power-forecasting\src\data_loader.py:23: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(


In [5]:
inspect_data(df)

Dataset shape: (2075259, 7)

Head:
                      Global_active_power  Global_reactive_power  Voltage  \
dt                                                                         
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
dt                                                                      
2006-12-16 17:24:00              18.4             0.0             1.0   
2006-12-16 17:25:00              23.0             0.0             1.0   
2006-12-16 17:26:00              23.0             0.0             2.0   
2006-12-16 17:27:00              23.0             0.0             

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0


## PreProcessing

In [6]:
df.columns = df.columns.str.lower()
df.columns

Index(['global_active_power', 'global_reactive_power', 'voltage',
       'global_intensity', 'sub_metering_1', 'sub_metering_2',
       'sub_metering_3'],
      dtype='object')

In [7]:
explain(df)

'The dataset contains 2,075,259 rows and 7 columns.\nMissing values: approximately 1.25% of the data.\n\nMain Features:\n- date: Date of measurement (dd/mm/yyyy).\n- time: Time of measurement (hh:mm:ss).\n- global_active_power (kW): Actual power consumed by appliances.\n- global_reactive_power (kW): Reactive (non-consumed) power.\n- voltage (V): Average voltage per minute.\n- global_intensity (A): Current intensity per minute.\n- sub_metering_1 (Wh): Kitchen appliances (dishwasher, oven, microwave).\n- sub_metering_2 (Wh): Laundry appliances (washing machine, dryer, fridge, lights).\n- sub_metering_3 (Wh): Water heater and air conditioner.\n\nNote: Unmeasured household consumption can be approximated by:\n(global_active_power*1000/60) – (sub_metering_1 + sub_metering_2 + sub_metering_3)'

In [8]:
describe_data(df) 


Statistical Description:
        global_active_power  global_reactive_power       voltage  \
count         2.049280e+06           2.049280e+06  2.049280e+06   
mean          1.091615e+00           1.237145e-01  2.408399e+02   
std           1.057294e+00           1.127220e-01  3.239987e+00   
min           7.600000e-02           0.000000e+00  2.232000e+02   
25%           3.080000e-01           4.800000e-02  2.389900e+02   
50%           6.020000e-01           1.000000e-01  2.410100e+02   
75%           1.528000e+00           1.940000e-01  2.428900e+02   
max           1.112200e+01           1.390000e+00  2.541500e+02   

       global_intensity  sub_metering_1  sub_metering_2  sub_metering_3  
count      2.049280e+06    2.049280e+06    2.049280e+06    2.049280e+06  
mean       4.627759e+00    1.121923e+00    1.298520e+00    6.458447e+00  
std        4.444396e+00    6.153031e+00    5.822026e+00    8.437154e+00  
min        2.000000e-01    0.000000e+00    0.000000e+00    0.000000e+00  

,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0


In [9]:
from summarytools import dfSummary # for summary of the data

dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,global_active_power[float64],Mean (sd) : 1.1 (1.1)min < med < max:0.1 < 0.6 < 11.1IQR (CV) : 1.2 (1.0),"4,186 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACDUlEQVR4nO3czUoCURyG8f/JkjFpBhF05yW0cOlFdLFt23stIkiOOGnKcNpkBFHqOVjznp7f2jmKPPgFvs57b0DTXf31AwBOcX3sBs65jpm1A8/fee83gdcCH34M1TnX6ff7D3me90IOX61Wz865R2JFrGOvqO08z3uTyWRTFMX2nIPLssym02lvsVi0zYxQEeXoW7+ZWVEU28Fg8BJwfifgGuALvkxBAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCwklrfqHqur4xs9w5F3oEQ8AwswuGWlXVzX6/vx+NRq1Wq3XWtuoBQ8A4uFiou93uOsuy2/F4vB0Oh8/nXs8QMD676Fu/mVm32w0dATZjCBjv+DIFCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCRf/u3SMyKUVVlYS0thQY5dWWFlJS2NDjVlaYWUlPY0N9SBiaYWVlYQ0PtRQLAmmJclQWRJMT5KhsiSYniRDPYhZEqzrOjd+GmuMk0ItyzI79+D1ep2ZmVVVlc3n89vfvD72vmez2V3MR4flcrl2zj2ZWdDHjn/q1Xv/7fP1Bhrv0Yey/P5JAAAAAElFTkSuQmCC"">","25,979(1.3%)"
2,global_reactive_power[float64],Mean (sd) : 0.1 (0.1)min < med < max:0.0 < 0.1 < 1.4IQR (CV) : 0.1 (1.1),532 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB+0lEQVR4nO3dwWrCQBRG4TuNSlSaIILufIQuXPoQfdhuu/dZRJAaSaqNyHRTS6G0mgm2+cP51s4gckgIeDPOe29A09399xcArtG59AHnXN/MeoH7l977feBa4NOvoTrn+uPx+DFJklHI5rvd7sU590SsqOvSFbWXJMlosVjs0zQ9VNk4y7J4uVyONptNz8wIFbVcvPWbmaVpephMJq8B+/cD1gDf8DAFCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCYQKCVedihLqdDp1zSxxzoVuwYFqMLMbhloURfd4PD7MZrMoiqJKZ1SdcaAazm4WalmWnTiOB/P5/DCdTl+qrudANXx101u/mdlwOAw9TM2MA9XwgYcpSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSLj5/1HrqDnKwhhLizQ21LqjLIyxtEtjQ60zysIYS/s0NtSzGqMsjLG0CA9TkECokND4W38oXn7RLleFmmVZXHXjPM9jM7OiKOL1ej34y/Wr1eq+7ssvtttt7px7NrOg9ajszXv/42/9Drbxn/IYp9qqAAAAAElFTkSuQmCC"">","25,979(1.3%)"
3,voltage[float64],Mean (sd) : 240.8 (3.2)min < med < max:223.2 < 241.0 < 254.2IQR (CV) : 3.9 (74.3),"2,837 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACqElEQVR4nO3dMW/aQBiA4e9KIDYgA6KBjZWhEgMjP6I/tj+hAysLKxObhUFGuBDAuFOltkra2IZw3+V99gun6CWKxXecybJMANt9uvcGgLd4uPcGXGSM8UWkVnD5Mcuy/TX34wJCvTJjjN/tdr8GQdApsn673W6MMd+I9U+Een21IAg6k8lk32q1DnkWxnHsTafTThRFNREh1N8Q6o20Wq1Dr9f7UWCpf/XNOICHKahAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAp/1WyZN06qIBMaYoj/CyTFBQrVIkiTV0+k0GgwGlUqlkmvy6hdXxwQJ1SLH4/HB87z6eDw+9Pv9Td71Lo8JEqqFGo1G0RFBEUfHBHmYggqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVOBw3ytKXMETXC6X6rX389ER6gvKXMFzOp28NE2/HA6H7yJS9CQp/kKoLyt8Bc9yuezM5/P6+Xyu3GpzHxGh/kORK3g2m42T5+rvjYcpqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBj1AdU/JWFWtvVCFUh5S9VcXmG1UI1SFlblWx/UYVQnVQiVtVrJ384mEKKjj7F7XEURIRjpNYx8lQyxwlEeE4iY2cDFVKHCUR4TiJjVwNVUSKHSUR4TiJjXiYggqEChUIFSpY/T8q31byvkrOCYjccFbA2lD5tpL3VXZOQOS2swL/DdUY44nI47Vf+A2CZrP5NBqNzr7vP+dZuFqtvMViUY/juBGG4THvC+92O09EJEkSLwzDuoa1Zdev1+um53n14XBo2u12rt+3iMh+v3+czWZPURR9NsZs864Xkecsy159g/wE91gbHrzETTUAAAAASUVORK5CYII="">","25,979(1.3%)"
4,global_intensity[float64],Mean (sd) : 4.6 (4.4)min < med < max:0.2 < 2.6 < 48.4IQR (CV) : 5.0 (1.0),221 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACFElEQVR4nO3cwWriUBiG4f8fW4kVEsRBd15CFy69iF5st7MUvBgh1IhprCKnm1oKpVVP6ky+zPusew5SXiJZ+HkIwYCm+/WvPwBwjptTf+DuPTPrRt6/CyFUkWeBd9+G6u694XD4kKbpIOby9Xr95O6PxIq6Tj1Ru2maDmazWZVl2faSi4uiSBaLxSDP

In [10]:
print(f'df - Number of duplicate Record: {df.duplicated().sum():,}')

df - Number of duplicate Record: 168,560


In [11]:
df[df.duplicated()]

,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3
dt,,,,,,,
2006-12-17 18:09:00,3.236,0.000,235.84,13.6,0.0,0.0,17.0
2006-12-18 00:04:00,0.206,0.000,245.98,0.8,0.0,0.0,0.0
2006-12-18 00:47:00,0.202,0.000,243.79,0.8,0.0,0.0,0.0
2006-12-18 01:54:00,0.276,0.096,243.61,1.2,0.0,0.0,0.0
2006-12-18 02:56:00,0.304,0.148,247.81,1.4,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2010-11-26 14:41:00,1.264,0.060,238.42,5.2,0.0,0.0,17.0
2010-11-26 15:18:00,1.272,0.060,239.76,5.2,0.0,0.0,18.0
2010-11-26 15:46:00,0.328,0.046,242.06,1.4,0.0,0.0,0.0


In [12]:
df.isnull().sum()

global_active_power      25979
global_reactive_power    25979
voltage                  25979
global_intensity         25979
sub_metering_1           25979
sub_metering_2           25979
sub_metering_3           25979
dtype: int64

In [13]:
percent_1 = df.isnull().sum()/df.isnull().count()*100
df.isnull().count()

global_active_power      2075259
global_reactive_power    2075259
voltage                  2075259
global_intensity         2075259
sub_metering_1           2075259
sub_metering_2           2075259
sub_metering_3           2075259
dtype: int64

In [14]:
# checking missing data
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum()/df.isnull().count()*100
percent_2 = (round(percent_1, 2)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data

,Total,%
global_active_power,25979,1.25
global_reactive_power,25979,1.25
voltage,25979,1.25
global_intensity,25979,1.25
sub_metering_1,25979,1.25
sub_metering_2,25979,1.25
sub_metering_3,25979,1.25


In [15]:
# finding all columns that have nan:
droping_list_all=[]
for j in range(0,7):
    if  df.iloc[:, j].notnull().any(): #if there are any null values in the column
        droping_list_all.append(j)
droping_list_all

[0, 1, 2, 3, 4, 5, 6]

In [16]:
def handle_missing_values(df):
    """
    Fill missing values using the column mean.
    """
    df = df.fillna(df.mean(numeric_only=True))
    return df

handle_missing_values(df)

,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0


In [17]:
def handle_missing_values(df):
    """
    Fill missing values using the column mean.
    """
    df = df.fillna(df.mean(numeric_only=True))
    return df

df = handle_missing_values(df)

In [18]:
df.isnull().sum()

global_active_power      0
global_reactive_power    0
voltage                  0
global_intensity         0
sub_metering_1           0
sub_metering_2           0
sub_metering_3           0
dtype: int64

### Time interval

In [19]:
interval = [df.index.min(), df.index.max()]
interval

[Timestamp('2006-12-16 17:24:00'), Timestamp('2010-11-26 21:02:00')]

In [20]:
# df["month"] = df.index.month

## outlires

In [21]:


def detect_outliers_zscore(df, cols, threshold=5):
    outliers = {}
    for col in cols:
        z_scores = zscore(df[col].dropna())
        outliers[col] = df[col][abs(z_scores) > threshold]
    return outliers

detect_outliers_zscore(df, df.columns)




{'global_active_power': dt
 2006-12-16 17:45:00    7.706
 2006-12-16 17:46:00    7.026
 2006-12-16 18:06:00    6.752
 2006-12-16 18:07:00    6.474
 2006-12-17 09:03:00    7.064
                        ...  
 2010-11-20 18:40:00    6.416
 2010-11-20 18:44:00    6.374
 2010-11-20 18:45:00    6.564
 2010-11-20 18:46:00    6.438
 2010-11-20 18:52:00    6.364
 Name: global_active_power, Length: 3445, dtype: float64,
 'global_reactive_power': dt
 2006-12-21 17:49:00    0.688
 2006-12-23 14:41:00    0.724
 2006-12-23 14:42:00    0.706
 2006-12-23 15:56:00    0.698
 2006-12-23 15:57:00    0.708
                        ...  
 2010-11-26 13:21:00    0.720
 2010-11-26 14:57:00    0.696
 2010-11-26 14:58:00    0.692
 2010-11-26 14:59:00    0.690
 2010-11-26 15:00:00    0.696
 Name: global_reactive_power, Length: 2380, dtype: float64,
 'voltage': dt
 2007-01-21 19:32:00    224.68
 2007-02-22 21:10:00    224.24
 2007-02-22 21:11:00    224.48
 2007-02-22 21:12:00    224.32
 2007-02-22 21:14:00    223

In [22]:
def detect_outliers_iqr(df, cols):
    outliers = {}
    for col in cols:
        Q1 = df[col].quantile(0.01)
        Q3 = df[col].quantile(0.99)
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
        outliers[col] = df[(df[col] < lower) | (df[col] > upper)][col]
    return outliers

detect_outliers_iqr(df, df.columns)

{'global_active_power': Series([], Name: global_active_power, dtype: float64),
 'global_reactive_power': dt
 2008-04-13 17:57:00    1.222
 2008-04-13 18:29:00    1.274
 2008-04-13 18:43:00    1.364
 2008-04-13 18:44:00    1.390
 2008-04-13 18:55:00    1.198
 2008-04-13 18:56:00    1.192
 2009-07-19 11:00:00    1.218
 2009-10-24 17:20:00    1.240
 Name: global_reactive_power, dtype: float64,
 'voltage': Series([], Name: voltage, dtype: float64),
 'global_intensity': Series([], Name: global_intensity, dtype: float64),
 'sub_metering_1': Series([], Name: sub_metering_1, dtype: float64),
 'sub_metering_2': Series([], Name: sub_metering_2, dtype: float64),
 'sub_metering_3': Series([], Name: sub_metering_3, dtype: float64)}

In [23]:
df.to_csv('../data/processed/preprocessed_data.csv', index=True)